<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [10]</a>'.</span>

## Ocean Stats

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings, yaml, os
from datetime import datetime
import getpass
import xarray as xr
from mom6_tools.stats import extract_time_series
warnings.filterwarnings("ignore")
from mom6_tools.DiagsCase import DiagsCase

Basemap module not found. Some regional plots may not function properly


In [3]:
# Make the graphs a bit prettier, and bigger
plt.style.use('ggplot')
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams.update({'font.size': 18})

In [4]:
# Read in the yaml file
diag_config_yml_path = "diag_config.yml"
diag_config_yml = yaml.load(open(diag_config_yml_path,'r'), Loader=yaml.Loader)
dcase = DiagsCase(diag_config_yml['Case'])

In [5]:
CASEROOT = dcase.get_value('CASEROOT')

In [6]:
# Parameters
sname = "placeholder-sname"
subset_kwargs = {}
product = "/glade/u/home/eromashkova/codes/mom6-tools/docs/source/examples/computed_notebooks/placeholder-sname/ocean_stats.ipynb"


In [7]:
rundir = dcase.get_value('RUNDIR')

In [8]:
DOUT_S = dcase.get_value('DOUT_S')

In [9]:
if DOUT_S:
  OUTDIR = dcase.get_value('DOUT_S_ROOT')+'/ocn/hist/'
else:
  OUTDIR = dcase.get_value('RUNDIR')

print('Output directory is:', OUTDIR)
print('Casename is:', dcase.casename)

Output directory is: /glade/scratch/gmarques/archive/g.e23_b15.GJRAv4.TL319_t232_zstar_N65.baseline.001/ocn/hist/
Casename is: g.e23_b15.GJRAv4.TL319_t232_zstar_N65.baseline.001


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [10]:
# create an empty class object
class args:
  pass

args.nw = 6 # requesting 6 workers
native = dcase.casename+diag_config_yml['Fnames']['native']
variables = ['thetaoga','soga']
extract_time_series(native, variables, dcase, args, OUTDIR)

TypeError: extract_time_series() missing 1 required positional argument: 'OUTDIR'

In [ ]:
header = ["Step", "Day","Truncs", "Energy/Mass",      
          "Maximum CFL", "Mean Sea Level", 
          "Total Mass", "Mean Salin", "Mean Temp",
         "Frac Mass Err", "Salin Err", "Temp Err"]
df = pd.read_csv(rundir+'/ocean.stats',  delimiter=',', 
                 usecols=(0,1,2,3,4,5,6,7,8,9,10,11),skiprows=(0,1),
                 names=header)#, skiprows=1) #, 
                # index_col=0,
                # dtype='unicode',
                # encoding = 'unicode_escape')

In [ ]:
df.head()

In [ ]:
# remove characters from each coloumn
for var in header[3::]:
    print(var)
    df[var] = df[var].str.replace(r'[^0-9.]+', '').astype(float)

df.head()

In [ ]:
# rename columns
new_header = ["Step", "Day","Truncs", "EnergyMass",      
          "MaximumCFL", "MeanSeaLevel", 
          "TotalMass", "MeanSalin", "MeanTemp",
         "FracMassErr", "SalinErr", "TempErr"]
for old, new in zip(header, new_header):
  df = df.rename(columns={old:new})

In [ ]:
df.head()

In [ ]:
units = ['nondim', 'days', 'nondim', 'm2 s-2',' nondim',
         'm', 'kg', 'PSU' , 'degC', 'nondim', 'PSU', 'degC']

data_vars = {}
for var, unit in zip(new_header, units):
  data_vars.update({var:(('time'), df[var], {"units" : unit})})

In [ ]:
ds = xr.open_dataset(rundir+"/ocean.stats.nc",use_cftime=False).rename({"Time" : "time"})

In [ ]:
variables = [
    'En',
    'Ntrunc',
    'Mass',
    'Mass_chg',
    'Mass_anom',
    'max_CFL_trans',
    'max_CFL_lin',
    'Salt',
    'Salt_chg',
    'Salt_anom',    
    'Heat',
    'Heat_chg',
    'Heat_anom',
    'age'
    
]

for v in variables:
  data_vars.update({v:(('time'), ds[v].values, {"units" : ds[v].units})})

In [ ]:
data_vars.update({"APE":(("time", "Interface"), 
                         ds.APE.values, {"units" : ds.APE.units})})
data_vars.update({"H0":(("time", "Interface"), 
                        ds.H0.values, {"units" : ds.H0.units})})
data_vars.update({"KE":(("time", "Layer"), 
                        ds.KE.values, {"units" : ds.KE.units})})
data_vars.update({"Mass_lay":(("time", "Layer"), 
                              ds.Mass_lay.values, {"units" : ds.Mass_lay.units})})

In [ ]:
#data_vars
time_units = "days since {}".format(dcase.get_value('RUN_STARTDATE')) 
attrs = {"units": time_units, "calendar" : "noleap"}
coords={"time": ("time", df["Day"], attrs),
        "Layer" : ("Layer", ds.Layer.values),
        "Interface" : ("Interface", ds.Interface.values)}

In [ ]:
# create dataset to store results
msg = " can be found at https://github.com/NCAR/mom6-tools"
attrs = {"description": "ocean stats time-series derived from ocean.stats",
         "casename" : dcase.casename,
         "caseroot" : CASEROOT,
         "author" : getpass.getuser(),
         "date" : datetime.now().isoformat(),
         "created_using" : "file.txt"}
stats = xr.decode_cf(xr.Dataset(data_vars=data_vars,
                   coords=coords,
                 attrs = attrs,
                ))

In [ ]:
if not os.path.isdir('ncfiles'):
      os.system('mkdir -p ncfiles')
                
stats.to_netcdf('ncfiles/{}_ocean.stats.nc'.format(dcase.casename))

In [ ]:
stats.time.plot()

### Truncations

In [ ]:
%matplotlib inline
stats.Truncs.plot();

### Maximum finite-volume CFL

In [ ]:
stats.max_CFL_trans.plot();

### Maximum finite-difference CFL

In [ ]:
stats.max_CFL_lin.plot();

### Maximum CFL

In [ ]:
stats.MaximumCFL.plot();

### Energy/Mass

In [ ]:
stats.EnergyMass.plot()

### Mean Sea Level

In [ ]:
stats.MeanSeaLevel.plot(); 

### Total Mass

In [ ]:
stats.TotalMass.plot(); 

### Mean Salinity

In [ ]:
stats.MeanSalin.plot(); 

### Mean Temperature

In [ ]:
stats.MeanTemp.plot(); 

### Total Energy

In [ ]:
stats.En.plot(); 

### Available Potential Energy

In [ ]:
stats.APE.sum(axis=1,keep_attrs=True).plot(); 

### Total Salt

In [ ]:
stats.Salt.plot(); 

### Total Salt Change between Entries

In [ ]:
stats.Salt_chg.plot(); 

### Anomalous Total Salt Change

In [ ]:
stats.Salt_anom.plot(); 

### Total Heat

In [ ]:
stats.Heat.plot(); 

### Total Heat Change between Entries

In [ ]:
stats.Heat_chg.plot(); 

### Anomalous Total Heat Change

In [ ]:
stats.Heat_anom.plot(); 

### Age

In [ ]:
stats.age.plot(); 